In [2]:
import json
import pandas as pd
import numpy as np

In [3]:
file_dir = '/Users/alexhatheway/Desktop/DataViz/Week_8_ETL/Movies-ETL/Data/'

#file_name = 'wikipedia-movies.json'

with open(f'{file_dir}wikipedia-movies.json', mode = 'r') as file:
    wiki_movies_raw = json.load(file)

len(wiki_movies_raw)

7311

In [5]:
# return first five records - or 6-11 in this case
wiki_movies_raw[6:11]

[{'url': 'https://en.wikipedia.org/wiki/American_Dream_(film)',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0099028/',
  'title': 'American Dream',
  'Directed by': ['Barbara Kopple',
   'Co-directors:',
   'Cathy Caplan',
   'Thomas Haneke',
   'Lawrence Silk'],
  'Produced by': ['Arthur Cohn', 'Barbara Kopple'],
  'Music by': 'Michael Small',
  'Cinematography': ['Tom Hurwitz', 'Mathieu Roberts', 'Nesya Shapiro'],
  'Edited by': ['Cathy Caplan', 'Thomas Haneke', 'Lawrence Silk'],
  'Productioncompany ': ['Cabin Creek',
   'Catholic Communication Campaign',
   'Channel 4 Films'],
  'Distributed by': 'Prestige Films',
  'Release date': ['October 6, 1990',
   '(',
   '1990-10-06',
   ')',
   '(',
   'New York Film Festival',
   ')'],
  'Running time': '100 minutes',
  'Country': ['United States', 'United Kingdom'],
  'Language': 'English'},
 {'url': 'https://en.wikipedia.org/wiki/American_Ninja_4:_The_Annihilation',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/t

In [6]:
# return last 5 records remember that :
wiki_movies_raw[-5:]

[{'url': 'https://en.wikipedia.org/wiki/Holmes_%26_Watson',
  'year': 2018,
  'imdb_link': 'https://www.imdb.com/title/tt1255919/',
  'title': 'Holmes & Watson',
  'Directed by': 'Etan Cohen',
  'Produced by': ['Will Ferrell',
   'Adam McKay',
   'Jimmy Miller',
   'Clayton Townsend'],
  'Screenplay by': 'Etan Cohen',
  'Based on': ['Sherlock Holmes',
   'and',
   'Dr. Watson',
   'by',
   'Sir Arthur Conan Doyle'],
  'Starring': ['Will Ferrell',
   'John C. Reilly',
   'Rebecca Hall',
   'Rob Brydon',
   'Steve Coogan',
   'Ralph Fiennes'],
  'Music by': 'Mark Mothersbaugh',
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Dean Zimmerman',
  'Productioncompanies ': ['Columbia Pictures',
   'Gary Sanchez Productions',
   'Mosaic Media Group',
   'Mimran Schur Pictures'],
  'Distributed by': 'Sony Pictures Releasing',
  'Release date': ['December 25, 2018',
   '(',
   '2018-12-25',
   ')',
   '(United States)'],
  'Running time': '90 minutes',
  'Country': 'United States',
  'Language

In [7]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory = False)

ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [10]:
kaggle_metadata.sample(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
31236,False,NaN,0,"[{'id': 16, 'name': 'Animation'}]",NaN,193652,tt0029350,en,An Optical Poem,A dance of shapes.,...,1937-01-01,0.0,6.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,An Optical Poem,False,7.2,13.0
23424,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,276122,tt3219678,en,A Brony Tale,Vancouver-based voice artist Ashleigh Ball has...,...,2014-07-08,0.0,79.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A film about men....who like My Little Pony,A Brony Tale,False,6.3,9.0
22691,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",NaN,295841,tt0093345,en,Killer Instinct,"For America, the Vietnam War is over. Johnny d...",...,1988-01-01,0.0,88.0,[],Released,NaN,Killer Instinct,False,6.0,1.0
6860,False,NaN,20000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...",NaN,2612,tt0100201,en,Mr. Destiny,Larry Burrows is unhappy and feels powerless o...,...,1990-10-12,15379253.0,110.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Larry Burrows Wished For It All... Until All T...,Mr. Destiny,False,6.1,60.0
30545,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,44511,tt0892332,en,Noodle,"At thirty-seven, Miri is a twice-widowed, El A...",...,2007-07-26,0.0,90.0,"[{'iso_639_1': 'he', 'name': 'עִבְרִית'}, {'is...",Released,NaN,Noodle,False,7.1,5.0
15000,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,52637,tt1149363,fr,Yuki & Nina,When Yuki finds out that her parents are separ...,...,2009-11-27,0.0,92.0,"[{'iso_639_1': 'ja', 'name': '日本語'}, {'iso_639...",Released,NaN,Yuki & Nina,False,5.5,1.0
39928,False,NaN,0,[],NaN,199929,tt2488740,en,Mister John,"After discovering his wife's infidelities, Ger...",...,2013-06-27,0.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Mister John,False,5.7,3.0
41452,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,328416,tt0091170,en,Hard Choices,A teenaged boy goes for a ride with his brothe...,...,1986-05-16,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Hard Choices,False,0.0,0.0
39193,False,NaN,0,"[{'id': 10402, 'name': 'Music'}, {'id': 99, 'n...",NaN,381080,tt5105736,en,The Art of Organized Noize,Organized Noize shaped the landscape of Hip Ho...,...,2016-03-15,0.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Art of Organized Noize,False,8.0,5.0
44846,False,"{'id': 315054, 'name': 'Dark World Collection'...",0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 9648, '...",NaN,51273,tt1746246,ru,Тёмный мир,"In Russia, the mystic student of linguistic Ma...",...,2010-10-07,0.0,100.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,Dark World,False,4.2,16.0


In [11]:
ratings.sample(10)

,userId,movieId,rating,timestamp
1769297,18366,5017,3.5,1447371443
16426053,170747,1073,3.5,1275023619
15195294,158133,2006,3.5,1357186854
10173553,104970,541,3.0,836445368
14418406,149856,5669,4.5,1177380520
6861331,70803,5816,3.5,1385598483
24764214,257460,4223,2.5,1231155966
11651191,120873,2959,5.0,975432793
19849198,206295,1198,4.0,1050518872
10283821,106146,267,1.0,831845048


In [14]:
# load raw data into dataframe and inspect it
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.tail(10)

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
7301,https://en.wikipedia.org/wiki/Aquaman_(film),2018.0,https://www.imdb.com/title/tt1477834/,Aquaman,James Wan,"[Peter Safran, Rob Cowan]","[David Leslie Johnson-McGoldrick, Will Beall]","[Geoff Johns, James Wan, Will Beall]","[Aquaman, by, Paul Norris, Mort Weisinger]","[Jason Momoa, Amber Heard, Willem Dafoe, Patri...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7302,https://en.wikipedia.org/wiki/Bumblebee_(film),2018.0,https://www.imdb.com/title/tt4701182/,Bumblebee,Travis Knight,"[Lorenzo di Bonaventura, Tom DeSanto, Don Murp...",NaN,NaN,"[Transformers, by, Hasbro]","[Hailee Steinfeld, John Cena, Jorge Lendeborg ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7303,https://en.wikipedia.org/wiki/Welcome_to_Marwen,2018.0,https://www.imdb.com/title/tt3289724/,Welcome to Marwen,Robert Zemeckis,"[Jack Rapke, Steve Starkey, Robert Zemeckis]","[Caroline Thompson, Robert Zemeckis]",NaN,NaN,"[Steve Carell, Leslie Mann, Diane Kruger, Merr...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7304,https://en.wikipedia.org/wiki/Second_Act_(film),2018.0,https://www.imdb.com/title/tt2126357/,Second Act,Peter Segal,"[Jennifer Lopez, Elaine Goldsmith-Thomas, Just...",NaN,NaN,NaN,"[Jennifer Lopez, Leah Remini, Vanessa Hudgens,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7305,https://en.wikipedia.org/wiki/Cold_War_(2018_f...,2018.0,https://www.imdb.com/title/tt6543652/,Cold War,Paweł Pawlikowski,"[Tanya Seghatchian, Ewa Puszczyńska]",NaN,NaN,NaN,"[Joanna Kulig, Tomasz Kot, Borys Szyc, Agata K...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zimna wojna
7306,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018.0,https://www.imdb.com/title/tt1255919/,Holmes & Watson,Etan Cohen,"[Will Ferrell, Adam McKay, Jimmy Miller, Clayt...",Etan Cohen,NaN,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7307,https://en.wikipedia.org/wiki/Vice_(2018_film),2018.0,https://www.imdb.com/title/tt6266538/,Vice,Adam McKay,"[Brad Pitt, Dede Gardner, Jeremy Kleiner, Kevi...",NaN,NaN,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7308,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018.0,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,Mimi Leder,Robert W. Cort,NaN,NaN,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7309,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018.0,https://www.imdb.com/title/tt7137380/,Destroyer,Karyn Kusama,"[Fred Berger, Phil Hay, Matt Manfredi]",NaN,NaN,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7310,https://en.wikipedia.org/wiki/Black_Mirror:_Ba...,2018.0,https://www.imdb.com/title/tt9495224/,Bandersnatch,David Slade,Russell McLean,NaN,NaN,NaN,"[Fionn Whitehead, Will Poulter, Craig Parkinso...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# load columns into list and inspect them
wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [19]:
# use list comprehension to filter out results that do not have a director or imdb link
#if ('Director' in movie or 'Directed by' in movie) and 'imdb_link' in movie

wiki_movies = [movie for movie in wiki_movies_raw \
              if ('Director' in movie or 'Directed by' in movie)\
               and 'imdb_link' in movie \
              and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [21]:
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018,https://www.imdb.com/title/tt1255919/,Holmes & Watson,Etan Cohen,"[Will Ferrell, Adam McKay, Jimmy Miller, Clayt...",Etan Cohen,NaN,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7072,https://en.wikipedia.org/wiki/Vice_(2018_film),2018,https://www.imdb.com/title/tt6266538/,Vice,Adam McKay,"[Brad Pitt, Dede Gardner, Jeremy Kleiner, Kevi...",NaN,NaN,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7073,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,Mimi Leder,Robert W. Cort,NaN,NaN,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7074,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018,https://www.imdb.com/title/tt7137380/,Destroyer,Karyn Kusama,"[Fred Berger, Phil Hay, Matt Manfredi]",NaN,NaN,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
        if len(alt_titles) > 0:
            movie['alt_titles'] = alt_titles
    return movie

In [22]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
6834,https://en.wikipedia.org/wiki/The_Insult_(film),2018,https://www.imdb.com/title/tt7048622/,The Insult,Ziad Doueiri,"[Rachid Bouchareb, Jean Bréhat, Julie Gayet, A...",NaN,NaN,NaN,"[Adel Karam, Kamel El Basha]",...,NaN,Case No. 23,NaN,NaN,NaN,قضية رقم ٢٣,Qadiyya raqm 23,NaN,NaN,NaN
7058,https://en.wikipedia.org/wiki/Capernaum_(film),2018,https://www.imdb.com/title/tt8267604/,Capernaum,Nadine Labaki,"[Michel Merkt, Khaled Mouzanar]","[Nadine Labaki, Jihad Hojaily, Michelle Keserw...","[Georges Khabbaz, Nadine Labaki, Michelle Kese...",NaN,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",...,NaN,NaN,NaN,NaN,NaN,کفرناحوم‎,NaN,NaN,NaN,NaN


In [23]:
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russian',
 'Screen story by',
 'Screenplay by',
 'Simplifie

In [25]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [26]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']